In [1]:
from grammar.generated.SqlBaseLexer import SqlBaseLexer
from grammar.generated.SqlBaseParser import SqlBaseParser
from grammar.generated.SqlBaseListener import SqlBaseListener

In [11]:
import antlr4
from antlr4 import InputStream
from antlr4 import RecognitionException

from antlr4.error.ErrorListener import ErrorListener
from antlr4.error.ErrorStrategy import BailErrorStrategy
from antlr4.error.Errors import ParseCancellationException

In [3]:
class RemoveIdentifierBackticks(antlr4.ParseTreeListener):
    @staticmethod
    def exitQuotedIdentifier(ctx):  # pylint: disable=invalid-name,unused-argument
        def identity(token):
            return token

        return identity

    @staticmethod
    def enterNonReserved(ctx):  # pylint: disable=invalid-name,unused-argument
        def add_backtick(token):
            return "`{0}`".format(token)

        return add_backtick


class ParseErrorListener(ErrorListener):
    def syntaxError(
            self, recognizer, offendingSymbol, line, column, msg, e
    ):  # pylint: disable=invalid-name,no-self-use,too-many-arguments
        raise SqlSyntaxError("Parse error", msg)


class UpperCaseCharStream:
    """
    Make SQL token detection case insensitive and allow identifier without
    backticks to be seen as e.g. column names
    """

    def __init__(self, wrapped):
        self.wrapped = wrapped

    def getText(self, interval, *args):  # pylint: disable=invalid-name
        if args or (self.size() > 0 and (interval.b - interval.a >= 0)):
            return self.wrapped.getText(interval, *args)
        return ""

    def LA(self, i: int):  # pylint: disable=invalid-name
        token = self.wrapped.LA(i)
        if token in (0, -1):
            return token
        return ord(chr(token).upper())

    def __getattr__(self, item):
        return getattr(self.wrapped, item)


class ExplicitBailErrorStrategy(BailErrorStrategy):
    """
    Bail Error Strategy throws a ParseCancellationException,
    This strategy simply throw a more explicit exception
    """
    def recover(self, recognizer, e: RecognitionException):
        try:
            super(ExplicitBailErrorStrategy, self).recover(recognizer, e)
        except ParseCancellationException:
            raise SqlParsingError from e


class EarlyBailSqlLexer(SqlBaseLexer):
    def recover(self, re: RecognitionException):
        raise SqlLexicalError from re


def build_parser(stream, strict_mode=False, early_bail=True):
    if not strict_mode:
        stream = UpperCaseCharStream(stream)
    if early_bail:
        lexer = EarlyBailSqlLexer(stream)
    else:
        lexer = SqlBaseLexer(stream)
    lexer.removeErrorListeners()
    lexer.addErrorListener(ParseErrorListener())
    token_stream = antlr4.CommonTokenStream(lexer)
    parser = SqlBaseParser(token_stream)
    parser.addParseListener(RemoveIdentifierBackticks())
    parser.removeErrorListeners()
    parser.addErrorListener(ParseErrorListener())
    if early_bail:
        parser._errHandler = ExplicitBailErrorStrategy()
    return parser


class SqlParsingError(Exception):
    pass


class SqlLexicalError(SqlParsingError):
    pass


class SqlSyntaxError(SqlParsingError):
    pass

In [18]:
def string_to_ast(string, rule, *, strict_mode=False, debug=False, early_bail=False):
    parser = build_string_parser(string, strict_mode, early_bail)
    tree = getattr(parser, rule)()
    if debug:
        print_tree(tree, printer=logging.warning)
    return tree


def build_string_parser(string, strict_mode=False, early_bail=True):
    string_as_stream = InputStream(string)
    parser = build_parser(string_as_stream, strict_mode, early_bail)
    return parser

In [19]:
def parse_sql(string, rule, converter, debug=False):
    tree = string_to_ast(string, rule, debug=debug)
    return converter(tree) if converter else tree


def parse_statement(string, converter=None, debug=False):
    return parse_sql(string, 'singleStatement', converter, debug)


In [23]:
def print_tree(tree, printer=print):
    for line in tree_to_strings(tree, indent=0):
        printer(line)


def tree_to_strings(tree, indent=0):
    symbol = ("[" + tree.symbol.text + "]") if hasattr(tree, "symbol") else ""
    node_as_string = type(tree).__name__ + symbol
    result = ["|" + "-" * indent + node_as_string]
    if hasattr(tree, 'children') and tree.children:
        for child in tree.children:
            result += tree_to_strings(child, indent + 1)
    return result


In [5]:
input_string = "SELECT * FROM table1"

In [24]:
tree = parse_statement(input_string).children[0]

ANTLR runtime and generated code versions disagree: 4.7.1!=4.9.3
ANTLR runtime and generated code versions disagree: 4.7.1!=4.9.3


In [25]:
print_tree(tree)

|StatementDefaultContext
|-QueryContext
|--QueryTermDefaultContext
|---QueryPrimaryDefaultContext
|----RegularQuerySpecificationContext
|-----SelectClauseContext
|------TerminalNodeImpl[SELECT]
|------NamedExpressionSeqContext
|-------NamedExpressionContext
|--------ExpressionContext
|---------PredicatedContext
|----------ValueExpressionDefaultContext
|-----------StarContext
|------------TerminalNodeImpl[*]
|-----FromClauseContext
|------TerminalNodeImpl[FROM]
|------RelationContext
|-------TableNameContext
|--------MultipartIdentifierContext
|---------ErrorCapturingIdentifierContext
|----------IdentifierContext
|-----------UnquotedIdentifierContext
|------------TerminalNodeImpl[table1]
|----------RealIdentContext
|--------TableAliasContext
|--QueryOrganizationContext
